# Tutorial SEGOMOE via web services

This notebook describes the usage of segomoe using web services exposed by WhatsOpt.

## Prerequisite

### wop (WhatsOpt command line interface)

You have to install the wop command version > 1.5.1:

```bash
pip install -U wop
```

You need to be logged in a WhatsOpt server either by using the <code>wop</code> command in a shell or the WhatsOpt API as follows:

In [1]:
from whatsopt.whatsopt_client import WhatsOpt
wop = WhatsOpt(url="https://ether.onera.fr/whatsopt")
ok = wop.login(echo=True)

# If you have any trouble with the previous command 
# then logout to ensure to start from a clean slate and retry
if not ok: 
    wop.logout() 

You have to set your API key.
You can get it in your profile page on WhatsOpt (https://ether.onera.fr/whatsopt).
Please, copy/paste your API key below then hit return (characters are hidden).
Your API key: ········
Successfully logged into WhatsOpt (https://ether.onera.fr/whatsopt)


To test if you are connected the following command should succeed

In [2]:
wop.check_versions()

WhatsOpt 1.7.0 requires wop >= 1.6.0
You use wop 1.6.0


### SMT (Surrogate Modeling Toolbox)

SMT is not required per se, but is used in this notebook to get optimized LHS sampling method.

```bash
pip install smt
```

## Optimization without constraint

### Objective function

First we define the objective function we want to minimize

In [3]:
import numpy as np

def f_obj(x):
    """
    Function Six-Hump Camel Back
    2 global optimum y_opt =-1.0316 located at x_opt = (0.089842, -0.712656) or (-0.089842, 0.712656)
    https://www.sfu.ca/~ssurjano/camel6.html
    """
    x_ = np.atleast_2d(x)
    x1 = np.array(x_)[:, 0]
    x2 = np.array(x_)[:, 1]
    val = 4*x1**2-2.1*x1**4+1./3.*x1**6+x1*x2-4*x2**2+4*x2**4
    return np.atleast_2d(val).T

### Optimization

First we create an optimization context to use the SEGOMOE optimizer with the design space <code>xlimits</code>.

In [4]:
from whatsopt.optimization import Optimization, ValidOptimumNotFoundError

xlimits = [[-3, 3], [-2, 2]]
optim = Optimization(xlimits)

We need to have an initial DOE (n_samples, nx) and the corresponding outputs y (n_samples, 1).

In [5]:
import numpy as np
# from smt.sampling_methods import LHS
# lhs = LHS(xlimits=np.array(xlimits), criterion='ese')
# xdoe = lhs(5)

xdoe = np.array([[-0.91502224,  1.89017506],
 [ 2.57253436,  0.83786997],
 [-2.32304511, -1.12821447],
 [ 1.65152355, -1.63067708],
 [ 0.0057155,  -0.31036381]])
ydoe = f_obj(xdoe)
print("Initial DOE")
print("xdoe={}".format(xdoe))
print("ydoe={}".format(ydoe))

Initial DOE
xdoe=[[-0.91502224  1.89017506]
 [ 2.57253436  0.83786997]
 [-2.32304511 -1.12821447]
 [ 1.65152355 -1.63067708]
 [ 0.0057155  -0.31036381]]
ydoe=[[37.11048572]
 [32.43194792]
 [16.82595609]
 [17.00496851]
 [-0.34983144]]


We initialize the optimizer with the inital DOE.

In [6]:
optim.tell_doe(xdoe, ydoe)

We trigger the optimization using the "ask and tell" interface.

In [7]:
# We loop using the iteration budget
n_iter = 15
for i in range(n_iter):
    x_suggested, status = optim.ask()
    print("{} x suggested = {} with status: {}".format(i, x_suggested, Optimization.STATUSES[status]))

    # compute objective function at the suggested point
    new_y = f_obj(np.atleast_2d(x_suggested))
    print("new y = {}".format(new_y))

    optim.tell(x_suggested, new_y)
    if optim.is_solution_reached():
        print("Solution is reached")
        break
        
    try:
        _, y = optim.get_result()
        print("y_opt_tmp = {}".format(y))
        print()
    except ValidOptimumNotFoundError:  # in case no point in doe respect constraints yet
        pass


x_opt, y_opt = optim.get_result()
print("Found minimum y_opt = {} at x_opt = {}".format(y_opt, x_opt))

0 x suggested = [0.5309094350878436, -0.39114866393954895] with status: valid point
new y = [[0.24206253]]
y_opt_tmp = -0.34983143933877303

1 x suggested = [-0.4466766342534265, -0.3472006102797744] with status: valid point
new y = [[0.44815116]]
y_opt_tmp = -0.34983143933877303

2 x suggested = [0.1730691365796534, -0.21557635754274626] with status: valid point
new y = [[-0.09662667]]
y_opt_tmp = -0.34983143933877303

3 x suggested = [0.0791748807123272, -0.3408412318094282] with status: valid point
new y = [[-0.41270041]]
y_opt_tmp = -0.41270041086970694

4 x suggested = [0.07925839855527265, -0.4089605334133059] with status: valid point
new y = [[-0.56447511]]
y_opt_tmp = -0.5644751069646898

5 x suggested = [0.10812241560218994, -0.7580541900927706] with status: valid point
new y = [[-1.0131991]]
y_opt_tmp = -1.0131990969429896

6 x suggested = [-0.4736146787626916, -1.586494282055799] with status: valid point
new y = [[16.8193071]]
y_opt_tmp = -1.0131990969429896

7 x suggested =

In [8]:
optim.get_result()

(array([ 0.09028977, -0.71245552]), -1.0316272516380196)

In [9]:
optim.get_history()

(array([[-0.91502224,  1.89017506],
        [ 2.57253436,  0.83786997],
        [-2.32304511, -1.12821447],
        [ 1.65152355, -1.63067708],
        [ 0.0057155 , -0.31036381],
        [ 0.53090944, -0.39114866],
        [-0.44667663, -0.34720061],
        [ 0.17306914, -0.21557636],
        [ 0.07917488, -0.34084123],
        [ 0.0792584 , -0.40896053],
        [ 0.10812242, -0.75805419],
        [-0.47361468, -1.58649428],
        [ 0.15547699, -0.69876421],
        [ 0.00934192, -0.70575352],
        [ 0.09124166, -0.71096074],
        [ 0.09013688, -0.7132663 ],
        [ 0.09048234, -0.7124495 ],
        [ 0.0903178 , -0.71298896],
        [ 0.09028977, -0.71245552],
        [ 0.08725179, -0.71233537]]), array([[37.11048572],
        [32.43194792],
        [16.82595609],
        [17.00496851],
        [-0.34983144],
        [ 0.24206253],
        [ 0.44815116],
        [-0.09662667],
        [-0.41270041],
        [-0.56447511],
        [-1.0131991 ],
        [16.8193071 ],
   

For convenience, the previous optimization loop is available as the <code>run</code> method of the optimization object.

In [10]:
# to reset the initial DOE, otherwise optimization will go on from previous state
# optim.tell_doe(xdoe, ydoe)

# run the optimization loop again
# optim.run(f_obj, 15)

## Optimization with constraints

### Objective and constraints functions

We define objective and constraints function and we build a grouped function which allows to evaluate all in one go.

In [11]:
# Objective
def G24(point):
    """
    Function G24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def f_grouped(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T

### Optimization

First we create an optimization context to use the SEGOMOE optimizer with the design space <code>xlimits</code> and the constraints specifications <code>cstr_specs</code>.

In [12]:
from whatsopt.optimization import Optimization, ValidOptimumNotFoundError

xlimits = [[0, 3], [0, 4]]
cstr_specs = 2*[{"type": '<', "bound": 0.0}]
optim = Optimization(xlimits, cstr_specs)

Constraint type can be iether <code><</code>, <code>=</code> or <code>></code>. A tolerance may be specified using the <code>tol</code> key. Constraints defaults are :

In [13]:
print(Optimization.DEFAULT_CSTR)

{'type': '<', 'bound': 0.0, 'tol': 0.0001}


In [14]:
import numpy as np
from smt.sampling_methods import LHS
lhs = LHS(xlimits=np.array(xlimits), criterion='ese')

xdoe = lhs(5)
ydoe = f_grouped(xdoe)
print("Initial DOE")
print("xdoe={}".format(xdoe))
print("ydoe={}".format(ydoe))

Initial DOE
xdoe=[[0.56130018 1.81171388]
 [1.92650023 2.60612985]
 [2.51029511 1.48097475]
 [1.20278127 0.12642011]
 [0.61950511 3.89770178]]
ydoe=[[-2.37301406 -1.4925358  -2.76666358]
 [-4.53263008  0.56603024 -1.35076916]
 [-3.99126986 -3.80090281 -0.70705137]
 [-1.32920138 -3.71248066 -0.40485263]
 [-4.51720689  0.43488684  0.61605257]]


In [15]:
optim.tell_doe(xdoe, ydoe)

In [16]:
optim.run(f_grouped, n_iter=20)

0 x suggested = [2.3376279820463717, 4.0] with status: valid point
new y = [[-6.33762798  0.75417324  0.85996393]]
y_opt_tmp = [-3.99126986 -3.80090281 -0.70705137]

1 x suggested = [3.0, 4.0] with status: valid point
new y = [[ -7. -16.   4.]]
y_opt_tmp = [-3.99126986 -3.80090281 -0.70705137]

2 x suggested = [1.8666481829594432, 4.0] with status: valid point
new y = [[-5.86664818  1.87607674  0.14099676]]
y_opt_tmp = [-3.99126986 -3.80090281 -0.70705137]

3 x suggested = [2.3550509817662024, 3.142629113017674] with status: valid point
new y = [[-5.49768009 -0.25570856  0.08755301]]
y_opt_tmp = [-3.99126986 -3.80090281 -0.70705137]

4 x suggested = [2.337805473968435, 3.1215621504914854] with status: valid point
new y = [[-5.45936762 -0.12576422 -0.01762423]]
y_opt_tmp = [-5.45936762 -0.12576422 -0.01762423]

5 x suggested = [2.3300516162694165, 3.170936100529563] with status: valid point
new y = [[-5.50098772e+00 -1.19006430e-02 -5.05787016e-03]]
y_opt_tmp = [-5.50098772e+00 -1.19006

(array([2.32951994, 3.1784958 ]),
 array([-5.50801574e+00,  4.82825441e-06,  1.51674423e-06]))

Note: if the suggested x has already been told or very close a previous suggestion (see <code>numpy.allclose</code>), the solution is considered to be reached.